In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2 as cv
from sklearn.model_selection import train_test_split

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:
# merge print into one folder:

roots = [
    "data_digits/hand",
    "data_digits/print",
] 

for root in roots:
    root = Path(root)

    for d in range(1, 10):
        (root / str(d)).mkdir(parents=True, exist_ok=True)
    
    files = root.rglob('*.jpg')

    for file in files:
        parent = file.parent.name

        file.rename(root / parent / file.name)



In [ ]:
# calculate mean and std

files = list(Path("data_digits/").rglob('*.jpg'))
print(len(files))

mean = 0
std = 0

for file in files:
    img = cv.imread(file, cv.IMREAD_GRAYSCALE)

    mean += img.mean()
    std += img.std()

mean /= len(files)
std /= len(files)

mean / 255, std / 255

In [ ]:
# split on train and test:
roots = [
    "data_digits/hand",
    "data_digits/print",
]

def create_split_cls_folders(root: Path, splits, classes):
    for split in splits:
        for cls in classes:
            (root / split / cls).mkdir(parents=True, exist_ok=True)

splits = ['train', 'test']
classes = [str(x) for x in range(1, 10)]

for root in roots:
    root = Path(root)
    files = list(root.rglob('*.jpg'))
    print(f"Have {len(files)}")

    create_split_cls_folders(root, splits, classes)

    train_files, test_files = train_test_split(files, test_size=0.10, random_state=1)

    for file in train_files:
        cls = file.parent.name
        file.rename(root / 'train' / cls / file.name)

    for file in test_files:
        cls = file.parent.name
        file.rename(root / 'test' / cls / file.name)


In [ ]:
folders = [x for x in Path('data_digits').rglob('*') if x.is_dir()]

for f in folders:
    if len(list(f.glob('*'))) == 0:
        f.rmdir()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Grayscale(),
    transforms.Resize((50, 50)),
    transforms.Normalize((0.082), (0.25696)), # Normalize to mean=0.1307, std=0.3081 (MNIST stats)
    #transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=10, translate=(0.15, 0.15), scale=(0.9, 1.1))
    #transforms.RandomPerspective(0.1, p=0.25),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Grayscale(),
    transforms.Resize((50, 50)),
    transforms.Normalize((0.082), (0.25696)), # Normalize to mean=0.1307, std=0.3081 (MNIST stats)
])

In [ ]:
train_dataset_hand = torchvision.datasets.ImageFolder(root='./data_digits/hand/train', transform=transform)
test_dataset_hand = torchvision.datasets.ImageFolder(root='./data_digits/hand/test', transform=transform_test)
train_dataset_print = torchvision.datasets.ImageFolder(root='./data_digits/print/train', transform=transform)
test_dataset_print = torchvision.datasets.ImageFolder(root='./data_digits/print/test', transform=transform_test)

train_dataset = torch.utils.data.ConcatDataset((train_dataset_hand, train_dataset_print))
test_dataset = torch.utils.data.ConcatDataset((test_dataset_hand, test_dataset_print))

# train_dataset = train_dataset_hand
# test_dataset = test_dataset_hand

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [ ]:
def calculate_params(model: nn.Module):
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return params

In [ ]:
import timm
timm.list_models('mobilenetv2*', pretrained=True)

In [ ]:
import timm
# model = timm.create_model('mobilenetv3_small_050.lamb_in1k', pretrained=True, num_classes=9, in_chans=1)
# print(calculate_params(model)) #577161

# model = timm.create_model('resnet10t.c3_in1k', pretrained=True, num_classes=9, in_chans=1)
# print(calculate_params(model))

model = timm.create_model('mobilenetv2_100.ra_in1k', pretrained=True, num_classes=9, in_chans=1)
print(calculate_params(model))


In [ ]:
from timm.models.mobilenetv3 import default_cfgs
default_cfgs['mobilenetv3_small_050']

In [ ]:

criterion = nn.CrossEntropyLoss()  # Standard loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=0.5, patience=5)


In [ ]:
def test_model(model, test_loader):

    model.eval()  # Set to evaluation mode
    correct = 0
    total = 0
    device = 'cuda'

    with torch.no_grad():  # No need to track gradients in evaluation
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")
    return 100 * correct / total


In [ ]:
num_epochs = 50  # You can increase this for better accuracy
device = 'cuda'
model.to(device)

best_score = 0

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero gradients from previous step
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    test_score = test_model(model, test_loader)

    if test_score > best_score:
        best_score = test_score
        torch.save(model.state_dict(), "best_model.pt")

    scheduler.step(running_loss, epoch=epoch)

print("Training complete!")


In [ ]:
model.load_state_dict(torch.load('best_model.pt', weights_only=True))
model.eval()

In [ ]:
test_model(model, test_loader)

In [ ]:
root = Path("data_digits/hand/test")

for digit in range(1, 10):
    (root / str(digit)).mkdir(parents=True, exist_ok=True)

In [ ]:
def viz(img):
    plt.figure()
    plt.imshow(img, cmap='gray')
    plt.show()



model.eval()
root = Path("data_digits/hand")
digits_files = list(root.glob('*.jpg'))


for file in digits_files:
    digit = cv.imread(file, cv.IMREAD_GRAYSCALE)

    inp = transform_test(digit).to(device)

    with torch.no_grad():
        output = model(inp.unsqueeze(0))
        _, pred = torch.max(output, 1)

    #viz(digit)
    #print(test_dataset.classes[pred])
    file.rename(file.parent / "test" / train_dataset.classes[pred] / file.name)

In [ ]:
root = Path("data_digits/hand")

for digit in range(1, 10):
    (root / str(digit)).mkdir()